# Irina Viner-Usmanova Rhythmic Gymnastics Centre

In [3]:
using WebIO

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-18320001915483595443\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-9317632583461618321\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-18320001915483595443\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [4]:
using Interact

In [5]:
using Khepri

In [6]:
render_size(800, 400) # Khepri backends size

(800, 400)

Avoid tests macro: `true` to skip tests, and `false` to run them.

In [7]:
avoid_tests = Parameter(true)

macro test(expr...)
  quote
    if !avoid_tests() 
        begin
            $(esc(expr...))
        end
    end
  end
end

@test (macro with 1 method)

In [8]:
avoid_tests(false)

false

## Sinusoidal Curves

 | Sinunoisal wave parameters | Other sinusoidal curves |
 |----------------------------|-------------------------|
 |<img src="./Figures/sin_wave.png" width="480"> | <img src="./Figures/sin_curves.png" width="350">|

`sinusoidal` parameters explained:
* a is the amplitude
* omega is the frequency
* fi is the phase

In [9]:
sinusoidal(a, omega, fi, x) = a*sin(omega*x+fi)

sinusoidal (generic function with 1 method)

In [10]:
sin_array_y(p, a, omega, fi, dist, n) = [p+vxy(i, sinusoidal(a, omega, fi, i)) for i in division(0, dist, n)]

sin_array_y (generic function with 1 method)

In [11]:
@test begin
    backend(notebook)
    new_backend()
    line(sin_array_y(u0(), 5, 1, 0, 50, 100))
end

data: []

layout: "layout with fields autosize, height, margin, scene, showlegend, and width"

Line(...)

Expected result:
<img src="./Plots/sin_array_y.png" width="600">

`damped_sin_wave`'s parameters explained
* a is the initial amplitude (the highest peak)
* b is the decay constant
* omega is the angular frequency

In [12]:
damped_sin_wave(a, d, omega, x) = a*exp(-(d*x))*sin(omega*x)

damped_sin_wave (generic function with 1 method)

In [13]:
damped_sin_array_z(p, a, d, omega, dist, n) = [p+vxz(i, damped_sin_wave(a, d, omega, i)) for i in division(0, dist, n)]

damped_sin_array_z (generic function with 1 method)

In [14]:
@test begin
    backend(notebook)
    new_backend()
    line(damped_sin_array_z(u0(), 5, 0.1, 1, 50, 100))
end

data: []

layout: "layout with fields autosize, height, margin, scene, showlegend, and width"

Line(...)

Expected result:
<img src="./Plots/damped_sin_array.png" width="600">

In [15]:
damped_sin_roof_pts(p, a_x, a_y, fi, decay, om_x, om_y, dist_x, dist_y, n_x, n_y) =
    map_division((x, y)->p+vxyz(x, y, sinusoidal(a_x, om_x, fi, x)+damped_sin_wave(a_y, decay, om_y, y)),
        0, dist_x, n_x,
        0, dist_y, n_y)

damped_sin_roof_pts (generic function with 1 method)

In [16]:
@test begin
    backend(meshcat)
    new_backend()
    @manipulate for ampx = widget(0:0.1:3, label="Amplitude x"),
                ampy = widget(0:0.1:20, label="Amplitude y")
            delete_all_shapes()
            surface_grid(damped_sin_roof_pts(u0(), ampx, ampy, 0, 0.1, 0.2, 1, 50, 100, 100, 200))
    end
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8700
└ @ MeshCat C:\Users\ines\.julia\packages\MeshCat\ECbzr\src\visualizer.jl:73


Khepri.MCATViewer(MeshCat Visualizer with path /meshcat at http://localhost:8700)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Amplitude x"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 31,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
16, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\ines\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\ines\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\ines\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\ines\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\ines\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00000000164fae10, Task (runnable) @0x00000000164fae10), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0\",\"0.1\",\"0.2\",\"0.3\",\"0.4\",\"0.5\",\"0.6\",\"0.7\",\"0.8\",\"0.9\",\"1.0\",\"1.1\",\"1.2\",\"1.3\",\"1.4\",\"1.5\",\"1.6\",\"1.7\",\"1.8\",\"1.9\",\"2.0\",\"2.1\",\"2.2\",\

Expected result:
<img src="./Plots/damped_surface.png" width="600">

In [17]:
damped_sin_roof_pts(p, h, a_x, a_y_min, a_y_max, fi, decay, om_x, om_y, dist_x, dist_y, n_x, n_y) =
    map_division((x, y) ->
                y <= d_i ?
                p + vxyz(x, 
                         -sin(y/d_i*(1*pi)), 
                         y*h/d_i + sin(x/dist_x*pi)sinusoidal(a_x, om_x, fi-y*pi/dist_y, x)*(y*a_x/d_i)) :
                p + vxyz(x,
                         y,
                         h + sin(x/dist_x*pi)*sinusoidal(a_x, om_x, fi-y*pi/dist_y, x) +
                         damped_sin_wave(a_y_max - (a_y_max-a_y_min)/dist_x*x, decay, om_y, y)),
             0, dist_x, n_x,
             0, dist_y, n_y)

damped_sin_roof_pts (generic function with 2 methods)

`d_i`is the distance between the pavillion starting point and the beginning of the dumped sine curve.

In [18]:
@test begin
    backend(meshcat)
    new_backend()
    d_i = 0.5
    @manipulate for ampx = widget(1:0.1:3, label="Amplitude x"),
                    ampy_min = widget(1:0.1:15, label="Minimum Amplitude y"),
                    ampy_max = widget(1:0.1:15, label="Maximum Amplitude y"),
                    h = widget(2:1:20, label="Height")
                delete_all_shapes()
                surface_grid(damped_sin_roof_pts(u0(), h, ampx, ampy_min, ampy_max, pi, 0.03, pi/50, pi/10, 60, 100, 120, 800))
    end
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8701
└ @ MeshCat C:\Users\ines\.julia\packages\MeshCat\ECbzr\src\visualizer.jl:73


Khepri.MCATViewer(MeshCat Visualizer with path /meshcat at http://localhost:8701)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Amplitude x"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 21,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
11, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\ines\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\ines\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\ines\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\ines\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\ines\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000037051cd0, Task (runnable) @0x0000000037051cd0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1.0\",\"1.1\",\"1.2\",\"1.3\",\"1.4\",\"1.5\",\"1.6\",\"1.7\",\"1.8\",\"1.9\",\"2.0\",\"2.1\",\"2.2\",\"2.3\",\"2.4\",\"2.5\",\"2.6\",\"2.7\",\"2.8\",\"2.9\",\"3.0\"],\"changes\":WebI

Expected result:
<img src="./Plots/roof_surface.png" width="600">

## Pavilion Dimensions

 | Roof dimentions explained | `d_i` explained |
 |---------------------------|-----------------|
 |<img src="./Figures/roof_dims.jpg" width="400"> | <img src="./Figures/roof_di.jpg" width="400"> |

__Double sinuoid parameters:__

* amp_x = amplitude of the sinusoid along the x axis
* amp_y_min_top = minimum amplitude of the damped sinusoid along the y axis for the top layer of the roof
* amp_y_max_top = maximum amplitude of the damped sinusoid along the y axis for the top layer of the roof
* amp_y_max_bottom = maximum amplitude of the damped sinusoid along the y axis for the bottom layer of the roof
* amp_y_min_bottom = minimum amplitude of the damped sinusoid along the y axis for the bottom layer of the roof
* fi = sinusoid's phase along the x axis
* decay = damped sinusoid's decay along the y axis
* om_x = frequency of the sinusoid in x
* om_y = frequency of the damped sinusoid in y

__Genaral pavilion parameters:__

* pav_width = pavilion's width (along x axis)
* pav_length = pavilion's lenght (along y axis)
* pav_height = pavilion's height (z axis)
* d_width = distance between roof layers in the x axis (top layer is wider)
* d_length = distance between roof layers in the y axis (top layer is longer)
* d_height = distance between roof layers in the z axis (height of the roof truss)
* d_i = distance before the damped sinusoid starts shaping the roof

__Construction elements parameters:__

* glass_panel_height = façade glass panel's height (width is defined by the number of points in the `damped_sinusoid` array)

In [41]:
amp_x = 2.5
amp_y_min_top = 4
amp_y_max_top = 11
amp_y_max_bottom = 10
amp_y_min_bottom = 3
fi = pi
decay = 0.03
om_x = pi/pav_width
om_y = 10*pi/pav_length
om_y_bottom = 10*pi/(pav_length - 2*d_length)
pav_width = 60
pav_length = 100
pav_height = 14
d_width = 1.5
d_length = 1
d_height = 3
d_i = 0.5
glass_panel_height = 1

1

## Roof Truss

### Basic truss elements

The following functions create truss elements:
* `free_node` and `fixed_node`create truss nodes
* `bar`creates truss bars
* `panel`creates truss panels

CAD version of truss elements:

In [20]:
free_node(pt) = sphere(pt, 0.1)
fixed_node(pt) = sphere(pt, 0.1)

fixed_node (generic function with 1 method)

In [21]:
truss_panel(ps) = surface_polygon(ps)

truss_panel (generic function with 1 method)

BIM families for truss elements:

In [21]:
free_node_fam = truss_node_family_element(default_truss_node_family(), support=Khepri.truss_node_support(), radius=0.1)
sup_node_fam = truss_node_family_element(default_truss_node_family(), support=Khepri.truss_node_support(ux=true, uy=true, uz=true), radius=0.1)

TrussNodeFamily("truss_node_family", 0.1, Khepri.TrussNodeSupport(true, true, true, false, false, false, Parameter{Bool}(false)), TrussNodeFamily("truss_node_family", 0.2, false, nothing, IdDict{Backend,Khepri.Family}(Unity => Khepri.UnityMaterialFamily("Default/Materials/Steel"),Khepri.MCATBackend{Khepri.MCATKey,String} => Khepri.BackendMaterialFamily{NamedTuple{(:uuid, :type, :metalness, :roughness, :side, :color),Tuple{String,String,Int64,Float64,Int64,String}}}((uuid = "1a3646d0-d1b2-11ea-0932-f59e46d99e04", type = "MeshStandardMaterial", metalness = 1, roughness = 0.5, side = 2, color = "0xFFFFFF")),AutoCAD => Khepri.ACADLayerFamily("TrussNodes", RGB{N0f8}(1.0,1.0,1.0), Parameter{Any}(nothing)),Khepri.POVRayBackend{Khepri.POVRayKey,Int64} => Khepri.BackendMaterialFamily{Khepri.POVRayMaterial}(Khepri.POVRayInclude("textures.inc", "texture", "Chrome_Metal")),Rhino => Khepri.RhinoLayerFamily("TrussNodes", RGB{N0f8}(1.0,1.0,1.0), Parameter{Any}(nothing))), IdDict{Backend,Any}()), IdDi

BIM version of truss elements:

In [22]:
fixed_node(p) = truss_node(p, family=sup_node_fam)
free_node(p) = truss_node(p, family=free_node_fam)

free_node (generic function with 1 method)

In [23]:
bar(p, q) = truss_bar(p, q)

bar (generic function with 1 method)

### Spatial truss

`truss_ptss`receives two sets of points, the bottom and top points defining the shape of the truss to create.
It returns the points defining the truss structure.

In [24]:
truss_ptss(bottom_ptss,top_ptss) =
    length(top_ptss) == length(bottom_ptss) ?
    vcat([[pts,qts] for (pts,qts) in zip(bottom_ptss,top_ptss)]...) :
    vcat([[pts,qts] for (pts,qts) in zip(top_ptss[1:end-1],bottom_ptss)]..., [top_ptss[end]])

truss_ptss (generic function with 1 method)

`truss`receives the type of truss to create, as well as both bottom and top points defining its shape.
It creates a 3D spatial truss.

In [25]:
truss(truss_type; bottom_ptss=planar(p=x(0)), top_ptss=planar(p=z(1))) =
    let ptss = truss_ptss(bottom_ptss, top_ptss)
      truss_type(ptss)
  end

truss (generic function with 1 method)

 | Vierendeel modular block front | Vierendeel modular block side |
 |----------------------------|-------------------------|
 |<img src="./Figures/front_vierendeel.png" width="350"> | <img src="./Figures/side_vierendeel.png" width="350">|

`vierendeel`creates a vierendeel truss structure.

In [26]:
vierendeel(ptss, first=true) =
    let ais = ptss[1],
        bis = ptss[2],
        cis = ptss[3],
        dis = ptss[4]
      (first ? fixed_node : free_node).(ais)
      free_node.(bis)
      bar.(ais, bis)
      bar.(ais, cis)
      bar.(bis, dis)
      bar.(ais[2:end], ais[1:end-1])
      bar.(bis[2:end], bis[1:end-1])
      if ptss[5:end] == []
          fixed_node.(cis)
          free_node.(dis)
          bar.(cis[2:end], cis[1:end-1])
          bar.(dis[2:end], dis[1:end-1])
          bar.(dis, cis)
      else
          vierendeel(ptss[3:end], false)
      end
    end

vierendeel (generic function with 2 methods)

In [27]:
@test begin
    backend(meshcat)
    new_backend()
    @manipulate for ampx = widget(1:0.1:3, label="Amplitude x"),
                    ampy_top_min = widget(3:0.1:15, label="Minimum Amplitude y"),
                    ampy_top_max = widget(3:0.1:15, label="Maximum Amplitude y"),
                    ampy_bottom_min = widget(1:0.1:12, label="Minimum Amplitude y"),
                    ampy_bottom_max = widget(1:0.1:12, label="Maximum Amplitude y"),
                    h = widget(2:1:20, label="Height")
                delete_all_shapes()
        truss(vierendeel, 
          top_ptss=damped_sin_roof_pts(u0(), pav_height,
                                          ampx, ampy_top_min, ampy_top_max,
                                          fi, decay, om_x, om_y,
                                          pav_width, pav_length-d_length, 50, 80),
          bottom_ptss=damped_sin_roof_pts(xy(d_width,d_length), pav_height-d_height,
                                         ampx, ampy_bottom_min, ampy_bottom_max,
                                         fi, decay, om_x, om_y_bottom,
                                         pav_width - d_width*2, pav_length-2*d_length, 50, 80))
    end
end

Expected result:
<img src="./Plots/truss_roof_meshcat.png" width="800">

In [28]:
@test begin
    backend(unity)
    delete_all_shapes()
    ground()
    truss(vierendeel, 
      top_ptss=damped_sin_roof_pts(u0(), pav_height,
                                      amp_x, amp_y_min_top, amp_y_max_top,
                                      fi, decay, om_x, om_y,
                                      pav_width, pav_length-d_length, 5, 8),
      bottom_ptss=damped_sin_roof_pts(xy(d_width,d_length), pav_height-d_height,
                                     amp_x, amp_y_min_bottom, amp_y_max_bottom,
                                     fi, decay, om_x, om_y_bottom,
                                     pav_width - d_width*2, pav_length-2*d_length, 5, 8))
#     render_dir("C:\\Users\\Renata\\Documents\\GitHub\\GymnasticsCenter_Moscow\\Plots")
#     render_view("truss_roof_unity")
end

Expected result:
<img src="./Plots/truss_roof_unity.png" width="800">

## Roof Surface

### Auxiliar functions

`tranpose_array`receives an array of arrays and transposes it.

In [22]:
transpose_array(arrays) =
    [[row[i] for row in arrays]
     for i in 1:length(arrays[1])]

transpose_array (generic function with 1 method)

`pts_distances`receives an array of points and returns an array with the distances between ordered points.

In [57]:
pts_distances(pts, last_pt=true) =
    let n =  last_pt ? 0 : 1
      [distance(p,q)
       for (p,q) in zip(pts,[pts[2:end]...,pts[1]])][1:end-n]
       end

pts_distances (generic function with 2 methods)

In [70]:
shape_grid_polygon_vertices(pts) =
    () -> pts

shape_grid_polygon_vertices (generic function with 1 method)

`itera_2triangs`receives an array of arrays of points. It returns the same points rearranged to create a triangular grid of points.

In [23]:
itera_2triangs(ptss) =
    vcat([vcat([[[p0,p1,p3],[p1,p2,p3]]
                for (p0,p1,p2,p3)
                in zip(pts0[1:end-1], pts1[1:end-1], pts1[2:end], pts0[2:end])]...)
         for (pts0, pts1) in zip(ptss[1:end-1], ptss[2:end])]...)

itera_2triangs (generic function with 1 method)

### Roof option 1: simple surfaces

In [46]:
@test begin
    backend(autocad)
    new_backend()
    top_roof = damped_sin_roof_pts(x(0), 15,
                                   3, 6, 9,
                                   fi, decay, om_x, om_y,
                                   pav_width, pav_length-d_length, 50, 100)
    bottom_roof = damped_sin_roof_pts(xy(d_width,d_length*1), 15-d_height,
                                      3, 6, 8,
                                      fi, decay, om_x, om_y_bottom,
                                      pav_width - d_width*2, pav_length-2*d_length, 50, 100)
    surface_grid(top_roof)
    surface_grid(bottom_roof)     
end

SurfaceGrid(...)

### Roof option 2: triangular roof panels

In [37]:
@test begin
    backend(autocad)
    new_backend()
    map(ps->surface_polygon(ps), itera_2triangs(top_roof))
    map(ps->surface_polygon(ps), itera_2triangs(bottom_roof))        
end

10000-element Array{SurfacePolygon,1}:
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 ⋮
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)

Expected result:
<img src="./Plots/roof_surf_polygons_cad.png" width="800">

### Roof lateral edges option 1: simple triangular panels

In [60]:
@test begin
    backend(autocad)
    new_backend()
    map(ps->surface_polygon(ps), itera_2triangs([top_roof[1],bottom_roof[1]]))
    map(ps->surface_polygon(ps), itera_2triangs([top_roof[end],bottom_roof[end]]))
    map(ps->surface_polygon(ps), itera_2triangs([transpose_array(top_roof)[end], transpose_array(bottom_roof)[end]]))
end

100-element Array{SurfacePolygon,1}:
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 ⋮
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)
 SurfacePolygon(...)

### Roof lateral edges option 2: Pinwheel tiling panels

`pattern`creates different patterns on parametric surfaces

In [50]:
pattern(fshape, pts...; args...) =
  fshape(pts...)(; (k=>v(pts) for (k,v) in args)...)

pattern (generic function with 1 method)

`pinwheel_rule`defines the basic subdivision rule of the pinwheel tiling.

 |    Pinwheel rule Tiling   |
 |---------------------------|
 |<img src="./Figures/tiling.png" width="400"> |
 source: https://mathematica.stackexchange.com/questions/55677/how-to-generate-nonperiodic-tilings

In [66]:
pinwheel_rule(pts, ratio1=0.383) =
    let dists = pts_distances(pts)
        max_dist = maximum(dists)
        min_dist = minimum(dists)
        ratio2 = 1 - ratio1
     if max_dist == dists[1]
         let pm = intermediate_loc(pts[1], pts[2], min_dist==dists[2] ? ratio1 : ratio2)
           [[pts[1],pm,pts[3]],
            [pts[2],pm,pts[3]]]
         end
     elseif max_dist == dists[2]
         let pm = intermediate_loc(pts[2], pts[3], min_dist==dists[1] ? ratio1 : ratio2)
           [[pts[1],pm,pts[2]],
            [pts[1],pm,pts[3]]]
         end
     else
        let pm = intermediate_loc(pts[1], pts[3], min_dist==dists[1] ? ratio1 : ratio2)
          [[pts[1],pm,pts[2]],
           [pts[2],pm,pts[3]]]
        end
     end
  end

pinwheel_rule (generic function with 2 methods)

`pinwheel_recursive_rule` repeats the basic subdivision rule `pinwheel_rule` several times (value set by the parameter `level`).

In [67]:
pinwheel_recursive_rule(pts, ratio1=0.383, level=1) =
    let qts = pinwheel_rule(pts, ratio1)
      level == 1 ?
      qts :
      [qts...,
       pinwheel_recursive_rule(qts[1], ratio1, level-1)...,
       pinwheel_recursive_rule(qts[2], ratio1, level-1)...]
     end

pinwheel_recursive_rule (generic function with 3 methods)

`pinwheel_tiling`receives a surface (described by its points) and creates the pinwheel tiling on it.

In [68]:
pinwheel_tiling(pts; ratio1=0.383, level=1) =
        map(surface_polygon, pinwheel_recursive_rule(pts, ratio1, level))

pinwheel_tiling (generic function with 1 method)

In [72]:
@test begin
    backend(autocad)
    new_backend()
    pinwheel_tiling.(pattern.(shape_grid_polygon_vertices,
                              itera_2triangs([top_roof[1],bottom_roof[1]])),
                     level=3)
    pinwheel_tiling.(pattern.(shape_grid_polygon_vertices,
                              itera_2triangs([top_roof[end],bottom_roof[end]])),
                     level=3)
    pinwheel_tiling.(pattern.(shape_grid_polygon_vertices,
                              itera_2triangs([transpose_array(top_roof)[end],transpose_array(bottom_roof)[end]])),
                     level=3)
end

100-element Array{Array{SurfacePolygon,1},1}:
 [SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...)]
 [SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...)]
 [SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...)]
 [SurfacePolygon(...), SurfacePolygon(...), SurfacePolygon(...), S

## Glass Façade

`splines4surf`maps splines over a matrix of points, to visualise surfaces in the notebook backend faster.

In [32]:
function splines4surf(mtx)
    [spline(pts) for pts in mtx]
    [spline(pts) for pts in transpose_array(mtx)]
end

splines4surf (generic function with 1 method)

`damped_sin_glass_wall`creates the pavillion's side glass wall matrixes

In [33]:
damped_sin_glass_wall(p, a_y, fi, decay, om_y, dist_y, dist_z, n_y, n_z) =
    map_division((y, z) ->
                    y <= d_i ?
                    p + vyz(-sin(y/d_i*(1*pi)), z) :
                    p + vyz(y,
                            z + (z/dist_z)*damped_sin_wave(a_y, decay, om_y, y)),
             0, dist_y, n_y,
             0, dist_z, n_z)

damped_sin_glass_wall (generic function with 1 method)

In [34]:
@test begin
    backend(autocad)
    new_backend()  
    west_glass_wall = damped_sin_glass_wall(x(d_width),
                                            amp_y_max_top,
                                            fi, decay, om_y,
                                            pav_length-2*d_length, pav_height-d_height, 40, 10)
    east_glass_wall = damped_sin_glass_wall(x(pav_width-d_width),
                                            amp_y_min_bottom,
                                            fi, decay, om_y,
                                            pav_length-2*d_length, pav_height-d_height, 40, 10)
#     delete_all_shapes()
    splines4surf(west_glass_wall)
    splines4surf(east_glass_wall)
end

Expected result:
<img src="./Plots/glass_lines_cad.png" width="800">

### Glass option 1: surface polygons

`itera_sweeked`creates the pavillion glass walls

In [35]:
itera_sweeked(ptss) =
    vcat([[let p23=intermediate_loc(p2, p3),
               p24=intermediate_loc(p2, p4)
             [p0,p1,p24,p23]
           end
          for (p0,p1,p2,p3,p4)
          in zip(pts0[1:end-1], pts1[1:end-1], pts1[2:end], pts0[2:end], pts2[2:end])]
         for (pts0, pts1, pts2)
         in zip(ptss[1:end-2], ptss[2:end-1],ptss[3:end])]...)

itera_sweeked (generic function with 1 method)

In [ ]:
@test begin
    backend(meshcat)
    new_backend()
    @manipulate for ampy_bottom_min = widget(1:0.1:12, label="Minimum Amplitude y"),
                    ampy_bottom_max = widget(1:0.1:12, label="Maximum Amplitude y"),
                    h = widget(2:1:20, label="Height")
                delete_all_shapes()
                west_glass_wall = damped_sin_glass_wall(x(d_width),
                                                        ampy_bottom_max,
                                                        fi, decay, om_y,
                                                        pav_length-2*d_length, h-d_height, 85, 20)
                east_glass_wall = damped_sin_glass_wall(x(pav_width-d_width),
                                                        ampy_bottom_min,
                                                        fi, decay, om_y,
                                                        pav_length-2*d_length, h-d_height, 85, 20)
                map(ps->surface_polygon(ps), itera_sweeked(west_glass_wall))
                map(ps->surface_polygon(ps), itera_sweeked(east_glass_wall))
    end
end

Expected result:
<img src="./Plots/glass_polys.png" width="800">

### Glass option 2: vertical BIM panels of diferent sizes

Given a list of points (closed polygon vertices), this function creates a polygonal glass panel surrounded by a thin metal framing all around:

In [37]:
framed_panel(pts)=
    begin
        panel(pts, family=default_panel_family())
        for (p,q) in zip(pts, [pts[2:end]...,pts[1]])
            free_column(p,q, family=frame_fam)
        end
    end

framed_panel (generic function with 1 method)

Frame BIM family

In [38]:
frame_width=0.1
frame_fam = column_family_element(default_column_family(), profile=rectangular_profile(frame_width, frame_width))

ColumnFamily("column_family", Khepri.RectangularPath(xyz(-0.05,-0.05,0.0), 0.1, 0.1), ColumnFamily("column_family", Khepri.RectangularPath(xyz(-0.1,-0.1,0.0), 0.2, 0.2), nothing, IdDict{Backend,Khepri.Family}(Unity => Khepri.UnityMaterialFamily("Default/Materials/Concrete"),Khepri.MCATBackend{Khepri.MCATKey,String} => Khepri.BackendMaterialFamily{NamedTuple{(:uuid, :type, :metalness, :roughness, :side, :color),Tuple{String,String,Int64,Float64,Int64,String}}}((uuid = "1a3646d0-d1b2-11ea-0932-f59e46d99e04", type = "MeshStandardMaterial", metalness = 1, roughness = 0.5, side = 2, color = "0xFFFFFF")),AutoCAD => Khepri.ACADLayerFamily("Column", RGB{N0f8}(1.0,1.0,1.0), Parameter{Any}(nothing)),Khepri.RadianceBackend{Khepri.RadianceKey,Int64,500} => Khepri.BackendMaterialFamily{RadianceMaterial}(RadianceMaterial("SheetMetal_80", "metal", 0.8, 0.8, 0.8, 0, 0, nothing, nothing)),Khepri.POVRayBackend{Khepri.POVRayKey,Int64} => Khepri.BackendMaterialFamily{Khepri.POVRayMaterial}(Khepri.POVRayIn

In [39]:
@test begin

    backend(meshcat)
    new_backend()
    @manipulate for radius = widget(1:1:5, label="Radius"),
                    sides = widget(1:1:10, label="Sides"),
                    angle = widget(1:1:20, label="Angle")
                    delete_all_shapes()        
                    framed_panel(regular_polygon_vertices(sides, x(-10), radius, angle))
                    framed_panel(regular_polygon_vertices(sides+2, x(0), radius, angle))
                    framed_panel(regular_polygon_vertices(sides+5, x(10), radius, angle))
                    end
  
end

Expected result:
<img src="./Plots/framed_panel.png" width="800">

Creates a vertical line of `framed_panels` given a list of points in z:

In [40]:
vertical_panel_line(pts, v, panel_width)=
    let pav_panel(p, q) = framed_panel([p, p+v*panel_width, q+v*panel_width, q])
        [pav_panel(p, q) for (p, q) in zip(pts[1:end-1], pts[2:end])]
    end

vertical_panel_line (generic function with 1 method)

In [41]:
@test begin
    backend(autocad)
    n_y = 40
    n_z = 10
    dist_y = pav_length-2*d_length
    dist_z = pav_height-d_height
    west_glass_wall = damped_sin_glass_wall(x(d_width),
                                            amp_y_max_top,
                                            fi, decay, om_y,
                                            dist_y, dist_z, n_y, n_z)
    east_glass_wall = damped_sin_glass_wall(x(pav_width-d_width),
                                            amp_y_min_bottom,
                                            fi, decay, om_y,
                                            dist_y, dist_z, n_y, n_z)
    panel_width = dist_y/n_y
    vertical_panel_line(west_glass_wall[1], vy(1), panel_width)
end

Expected result:
<img src="./Plots/first_row_panels.png" width="800">

Same function, but vector `v` now already considers the panel width, it's not a normal vector anymore:

In [42]:
vertical_panel_line(pts, v)=
    let pav_panel(p, q) = framed_panel([p, p+v, q+v, q])
        [pav_panel(p, q) for (p, q) in zip(pts[1:end-1], pts[2:end])]
    end

vertical_panel_line (generic function with 2 methods)

Makes a `vertical_panel_line` for all lines in the matrix:

In [43]:
vertical_panel_lines(mtx)=
    let v = mtx[2][1]-mtx[1][1]
        [vertical_panel_line(pts, v) for pts in mtx]
    end

vertical_panel_lines (generic function with 1 method)

In [44]:
@test begin
    backend(autocad)
    n_y = 40
    n_z = 10
    dist_y = pav_length-2*d_length
    dist_z = pav_height-d_height
    east_glass_wall = damped_sin_glass_wall(x(pav_width-d_width),
                                            amp_y_min_bottom,
                                            fi, decay, om_y,
                                            dist_y, dist_z, n_y, n_z)
    delete_all_shapes()
    vertical_panel_lines(west_glass_wall)
end

Expected result:
<img src="./Plots/weast_facade_panel.png" width="800">

### Glass option 3: same size vertical BIM panels

Predicate functions: is current `z` is above the limit ?

In [45]:
is_above_z_limit(z, z_lim)= z >= z_lim

is_above_z_limit (generic function with 1 method)

`vert_pts` creates the orgin points for all vertical panels above p. Stops creating points in z when the set limit is reached:

In [46]:
vert_pts(p, panel_height, z_lim) = is_above_z_limit(p.z, z_lim) ? [] : [p, vert_pts(p+vz(panel_height), panel_height, z_lim)...]

vert_pts (generic function with 1 method)

`damped_sin_glass_wall` creates a vertical line of panels for all points between `p` and `p+vy(dist_y)`. The `z_limit` for all vertical lines of panels is defined by the`z` value of the `damped_sin_wave` at any moment in the `y` progression.

In [47]:
damped_sin_glass_wall(p, a_y, fi, decay, om_y, dist_y, dist_z, n_y, n_z, panel_height) =
    let pts = map_division(y -> p+vy(y), 0, dist_y, n_y)
    zs = map_division(y -> dist_z+damped_sin_wave(a_y, decay, om_y, y), 0, dist_y, n_y) 
    [vert_pts(p, panel_height, z_lim) for (p, z_lim) in zip(pts, zs)]
    end

damped_sin_glass_wall (generic function with 2 methods)

 `glass_panel_height` defines façade glass panel's height (width is defined by the number of points in the `damped_sinusoid` array)

In [48]:
@test begin
    backend(autocad)
    glass_panel_height = 0.5
    n_y = 40
    n_z = 10
    dist_y = pav_length-2*d_length
    dist_z = pav_height-d_height
    west_glass_wall = damped_sin_glass_wall(x(d_width),
                                            amp_y_max_top,
                                            fi, decay, om_y,
                                            dist_y, dist_z, n_y, n_z)
    east_glass_wall = damped_sin_glass_wall(x(pav_width-d_width),
                                            amp_y_min_bottom,
                                            fi, decay, om_y,
                                            dist_y, dist_z, n_y, n_z)
    delete_all_shapes()
    vertical_panel_lines(west_glass_wall)
end

Expected result:
<img src="./Plots/planels_2d_try_cad.png" width="800">

### Glass option 4: intercalated rows of vertical BIM panels

Diffrent starting point for vertical panels in the odd case:

In [49]:
vert_pts_odd(p, panel_height, z_lim) = [p, vert_pts(p+vz(panel_height/2), panel_height, z_lim)...]

vert_pts_odd (generic function with 1 method)

Interweaving 2 different functions depending on `count` being pair or odd:

In [50]:
f_weave(f1, f2, count) = isodd(count) ? f1 : f2

f_weave (generic function with 1 method)

Interweaved version of the `damped_sin_glass_wall` function. Different starting height for the first panel of odd rows.

In [51]:
damped_sin_glass_wall(p, a_y, fi, decay, om_y, dist_y, dist_z, n_y, n_z, panel_height) =
    let pts = map_division(y -> p+vy(y), 0, dist_y, n_y)
        zs = map_division(y -> dist_z+damped_sin_wave(a_y, decay, om_y, y), 0, dist_y, n_y) 
        ns = 1:length(zs)
     [f_weave(vert_pts_odd(p, panel_height, z_lim),vert_pts(p, panel_height, z_lim), count)	for (p, z_lim, count) in zip(pts, zs, ns)]
    end

damped_sin_glass_wall (generic function with 2 methods)

In [52]:
@test begin
    backend(autocad)
    glass_panel_height = 1
    n_y = 40
    n_z = 10
    dist_y = pav_length-2*d_length
    dist_z = pav_height-d_height
    west_glass_wall = damped_sin_glass_wall(x(d_width),
                                            amp_y_max_top,
                                            fi, decay, om_y,
                                            dist_y, dist_z, n_y, n_z)
    delete_all_shapes()
    vertical_panel_lines(west_glass_wall)
end

Expected result:
<img src="./Plots/panels_weave.png" width="800">

## Interior Walls

## Backend specifics

Ground:

In [53]:
ground_fam = slab_family_element(default_slab_family())
ground() =
    let x = 1000
        y = 1000
      slab(closed_polygonal_path([xy(-x,y), xy(x,y), xy(x,-y), xy(-x,-y)]), level(-0.05), ground_fam)
    end

ground (generic function with 1 method)

BIM family materials for Unity backend:

In [54]:
set_backend_family(default_truss_bar_family(), unity, unity_material_family("Default/Materials/Aluminum"))
set_backend_family(free_node_fam, unity, unity_material_family("Default/Materials/Aluminum"))
set_backend_family(sup_node_fam, unity, unity_material_family("materials/metal/Copper"))
# set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/White")) # -- white ground
set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/WhiteUnlit")) # -- white ground no shadows
set_backend_family(default_panel_family(), unity, unity_material_family("Default/Materials/GlassBlue"))
set_backend_family(frame_fam, unity, unity_material_family("Default/Materials/Steel"))

IdDict{Backend,Any} with 0 entries

## Complete Building